# Demo chatzilla

## Imports

first 7 lines are for relative imports using jupyter notebooks

In [1]:
import sys
import os
# Add the parent directory to the path
notebook_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(notebook_dir, '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

import json
import datetime
import requests
from dotenv import load_dotenv
from pydantic import BaseModel, ValidationError, Field
from typing import Dict, List, Literal, Union
from chatzilla import zillaping, PromptOllama, ChatOllama
from chatzilla.logger import save_history_to_json

In [3]:
load_dotenv() # rename .env.sample to .env
ollama_url_ping = os.getenv("OLLAMA_URL") # http://localhost:11434
ollama_url_prompt = os.getenv("OLLAMA_GEN") # http://localhost:11434/api/generate
ollama_url_chat = os.getenv("OLLAMA_CHAT") # http://localhost:11434/api/chat
model = os.getenv("DEFAULT_MODEL") # llama3.1

## Ping ollama

In [4]:
zillaping(ollama_url_ping)

'Ollama is running'

## Prompt

Single prompt without any chat history

In [4]:
prompt = "tell me a bill burr joke"
joke = PromptOllama(prompt, model, ollama_url_prompt)
print(joke)

Here's one:

"You know what's wild? We spend the first year of a child's life teachin' 'em to walk and talk, and the rest of their lives tellin' 'em to shut up and sit down. That's just backwards, ain't it?"


## Chat with history

In [5]:
chat = ChatOllama(ollama_url_chat, model)
msg1 = chat.begin(prompt)
msg2 = chat.next("make the joke edgy")

print(f"first message received:\n\t{msg1}\n\n")
print(f"second message received:\n\t{msg2}")

first message received:
	Here's one:

"You know what's wild? We spend the first year of a child's life teachin' 'em to walk and talk, and the rest of their lives tellin' 'em to shut up and sit down. That's just good business sense."

(Note: Bill Burr is known for his edgy, sarcastic humor, so keep in mind that this joke might not be to everyone's taste!)


second message received:
	Here's a more edgy Bill Burr-style joke:

"You know what they say about kids these days? They're all entitled and selfish. But you know who the real problem is? It's their parents, man. They're just as bad. 'My child was bullied... my child this... my child that.' No, your kid got pushed around on the playground because he's a little piece of garbage, okay? That's what happened. You didn't raise a superhero, you raised a Twitter account."

(again, keep in mind that Bill Burr's humor is often not for everyone!)


## save chat history

In [ ]:
save_history_to_json(chat.history())

## Validate output

Check out this [blog post](https://ollama.com/blog/structured-outputs) for more information on forcing ollama to return a specific response format

In [7]:
class Country(BaseModel):
  name: str = Field(..., description="The name of the country")
  capital: str = Field(..., description="The capital city of the country")
  provinces: List[str] = Field(..., description="A list of all province names in the country")
  languages: List[str] = Field(..., description="A list of official languages spoken in the country")

format = Country.model_json_schema()

In [13]:
prompt = "tell me about Canada"
info = PromptOllama(prompt, model, ollama_url_prompt, format)
print(info)

{ "name": "Canada", "capital": "Ottawa", "provinces": ["British Columbia","Alberta","Saskatchewan","Manitoba","Ontario","Quebec","Nova Scotia","New Brunswick","Prince Edward Island","Newfoundland and Labrador","Yukon","Northwest Territories","Nunavut"] , "languages": ["English, French" ] }

   	    		       			


use [pydantic](https://docs.pydantic.dev/latest/) to validate the response schema

In [14]:
try:
    parsed = json.loads(info)
    validated = Country(**parsed)
    print(validated)
except (json.JSONDecodeError, ValidationError) as e:
    print("Validation failed:", e)

name='Canada' capital='Ottawa' provinces=['British Columbia', 'Alberta', 'Saskatchewan', 'Manitoba', 'Ontario', 'Quebec', 'Nova Scotia', 'New Brunswick', 'Prince Edward Island', 'Newfoundland and Labrador', 'Yukon', 'Northwest Territories', 'Nunavut'] languages=['English, French']


## Tools

ollama supports the use of tools, [click here](https://ollama.com/blog/tool-support) for more information

Example below uses two tools:
1. Simple python function to get the current time
2. API request to [Open-meteo](https://open-meteo.com/en/docs?latitude=44.3001&longitude=-78.3162) to get weather data

In [5]:
import openmeteo_requests

import datetime
import pandas as pd
import requests_cache
from retry_requests import retry

In [6]:
def get_current_time() -> str:
    return f"{datetime.datetime.now()}"

In [12]:
def get_current_weather(latitude:float=44.3001, longitude:float=-78.3162) -> Dict[Literal['date'], Dict[str, Union[int, float]]]:
    retry_session = retry(retries = 5, backoff_factor = 0.2)
    openmeteo = openmeteo_requests.Client(session = retry_session)

    # Make sure all required weather variables are listed here
    # The order of variables in hourly or daily is important to assign them correctly below
    url = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "hourly": "temperature_2m"
    }
    responses = openmeteo.weather_api(url, params=params)

    # Process first location. Add a for-loop for multiple locations or weather models
    response = responses[0]

    # Process hourly data. The order of variables needs to be the same as requested.
    hourly = response.Hourly()
    hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()

    hourly_data = {"date": pd.date_range(
        start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
        end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
        freq = pd.Timedelta(seconds = hourly.Interval()),
        inclusive = "left"
    )}

    hourly_data["temperature_2m"] = hourly_temperature_2m

    hourly_dataframe = pd.DataFrame(data = hourly_data)
    return hourly_dataframe

In [13]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather for a given latitude and longitude",
            "parameters": {
                "type": "object",
                "properties": {
                    "latitude": {
                        "type": "number",
                        "description": "Latitude of the location"
                    },
                    "longitude": {
                        "type": "number",
                        "description": "Longitude of the location"
                    }
                },
                "required": ["latitude", "longitude"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_current_time",
            "description": "Get the current time of day",
            "parameters": {
                "type": "object",
                "properties": {
                    "timestamp": {
                        "type": "number",
                        "description": "current time"
                    }
                },
                "required": ["timestamp"]
            }
        }
    }
]

use tool call to request weather data with `get_current_time` and `get_current_weather` functions.

In [14]:
h = {"Content-Type":"application/json"}
d = {
    "model":model,
    "messages": [{
        "role": "user", 
        "content": "What is the weather like right now in Peterborough Ontario (use Celsius)? Additionally, return the time of day."
    }],
    "stream":False,
    "tools":tools
}

response_json = requests.post(ollama_url_chat, headers=h, json=d).json()
response_json

{'model': 'llama3.1',
 'created_at': '2025-05-19T05:09:45.5538369Z',
 'message': {'role': 'assistant',
  'content': '',
  'tool_calls': [{'function': {'name': 'get_current_weather',
     'arguments': {'latitude': 44.15, 'longitude': -78.2056}}},
   {'function': {'name': 'get_current_time', 'arguments': {}}}]},
 'done_reason': 'stop',
 'done': True,
 'total_duration': 6133074600,
 'load_duration': 16704500,
 'prompt_eval_count': 236,
 'prompt_eval_duration': 336864300,
 'eval_count': 132,
 'eval_duration': 5778981900}

parse output from assistant, note the `tool_calls` field in the json response above. This can now be used to call python functions using those arguments

In [15]:
for func in response_json['message']['tool_calls']:
    match func['function']['name']:
        case 'get_current_weather':
            current_weather = get_current_weather(func['function']['arguments']['latitude'], func['function']['arguments']['longitude'])
        case 'get_current_time':
            current_time = get_current_time()

include chat history and send another request to include the data returned from python functions into the chat history

first message must be using `user` role and the second message must be the `tool` role, also include the tools list into the body of request

In [16]:
h = {"Content-Type":"application/json"}
d = {
    "model":model,
    "messages": [
        {
            "role": "user", 
            "content": "What is the weather like right now in Peterborough Ontario (use Celsius)? Additionally, return the time of day."
        },
        {
            "role": "tool",
            "content": f"current time:\n{current_time}\nweather data:\n{current_weather}"
        }
    ],
    "tools":tools
}

response_json = requests.post(ollama_url_chat, headers=h, json=d).json()
response_json

{'model': 'llama3.1',
 'created_at': '2025-05-19T05:09:50.5857908Z',
 'message': {'role': 'assistant',
  'content': 'Based on the weather data, it appears that at this moment (01:09:46), the temperature in Peterborough, Ontario is approximately 10.87°C. The time of day is currently 01:09 AM.'},
 'done_reason': 'stop',
 'done': True,
 'total_duration': 2309334200,
 'load_duration': 16198300,
 'prompt_eval_count': 374,
 'prompt_eval_duration': 385715900,
 'eval_count': 47,
 'eval_duration': 1904814700}